In [31]:
import numpy as np
from intfeat import StrumLiouvilleTransformer
import optuna
from ucimlrepo import fetch_ucirepo
from sklearn.linear_model import RidgeCV, Ridge
from sklearn.metrics import root_mean_squared_error
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import SplineTransformer, KBinsDiscretizer, StandardScaler

In [32]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

# Generate dataset

In [3]:
dataset = fetch_ucirepo(id=332)
float_feat_cols = [
 'data_channel_is_world',
 'weekday_is_sunday',
 'kw_avg_max',
 'is_weekend',
 'kw_max_avg',
 'global_rate_positive_words',
 'abs_title_sentiment_polarity',
 'self_reference_avg_sharess',
 'max_positive_polarity',
 'avg_positive_polarity']
int_feat_cols = ['kw_min_max', 'n_tokens_title']
all_features = float_feat_cols + int_feat_cols

X = dataset.data.features
y = dataset.data.targets

X.columns = X.columns.map(lambda col: col.strip())
X = X[all_features]
X = X.astype({col: np.int32 for col in int_feat_cols})

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
y_train = y_train.values.ravel()
y_test = y_test.values.ravel()

In [5]:
num_funcs = 5

In [ ]:
def make_spline_pipeline(knots='uniform', degree=3, num_funcs=num_funcs):
    n_knots = num_funcs - (degree - 1)
    return make_pipeline(
        SplineTransformer(n_knots=n_knots, knots=knots, degree=degree),
        RidgeCV(alphas=np.geomspace(1e-3, 1e3, 20))
    )

In [ ]:
def make_bins_pipeline(strategy='uniform', num_funcs=num_funcs):
    return make_pipeline(
        KBinsDiscretizer(n_bins=num_funcs, encode='onehot', strategy=strategy),
        RidgeCV(alphas=np.geomspace(1e-3, 1e3, 20))
    )

In [ ]:
def make_sl_pipeline(curvature_gamma=1., spline_degree=3, num_funcs=num_funcs):
    spline_n_knots = num_funcs - (spline_degree - 1)
    spline_knots = 'quantile'
    return make_pipeline(
        make_column_transformer(
            (
                SplineTransformer(n_knots=spline_n_knots, knots=spline_knots, degree=spline_degree),
                float_feat_cols
            ),
            (
                StrumLiouvilleTransformer(num_funcs=num_funcs, curvature_gamma=curvature_gamma),
                int_feat_cols
            )
        ),
        RidgeCV(alphas=np.geomspace(1e-3, 1e3, 20))
    )

In [9]:
pipeline = make_spline_pipeline('uniform').fit(X_train, y_train)
root_mean_squared_error(y_test, pipeline.predict(X_test))

10895.498751756086

In [10]:
pipeline = make_spline_pipeline('quantile').fit(X_train, y_train)
root_mean_squared_error(y_test, pipeline.predict(X_test))

10883.391677279977

In [11]:
pipeline = make_bins_pipeline('uniform').fit(X_train, y_train)
root_mean_squared_error(y_test, pipeline.predict(X_test))

10949.338290115285

In [12]:
pipeline = make_bins_pipeline('quantile').fit(X_train, y_train)
root_mean_squared_error(y_test, pipeline.predict(X_test))

10882.599855280469

In [13]:
pipeline = make_sl_pipeline().fit(X_train, y_train)
root_mean_squared_error(y_test, pipeline.predict(X_test))

10884.36357164014

# HPO

In [46]:
class SplineHpo:
    def search_space(self, trial: optuna.Trial):
        knots = trial.suggest_categorical('knots', ['uniform', 'quantile'])
        degree = trial.suggest_int('degree', 1, 3)
        num_funcs = trial.suggest_int('num_funcs', 1 + degree, 10)
        alpha = trial.suggest_float('alpha', 1e-3, 1e3, log=True)

        return {
            'knots': knots,
            'degree': degree,
            'num_funcs': num_funcs,
            'alpha': alpha
        }

    def pipeline(self, knots, degree, num_funcs, alpha):
        n_knots = num_funcs - (degree - 1)
        return make_pipeline(
            SplineTransformer(n_knots=n_knots, knots=knots, degree=degree),
            Ridge(alpha=alpha)
        )

In [54]:
class BinsHPO:
    def search_space(self, trial: optuna.Trial):
        strategy = trial.suggest_categorical('strategy', ['uniform', 'quantile'])
        num_funcs = trial.suggest_int('num_funcs', 2, 10)
        alpha = trial.suggest_float('alpha', 1e-3, 1e3, log=True)

        return {
            'strategy': strategy,
            'num_funcs': num_funcs,
            'alpha': alpha
        }

    def pipeline(self, strategy, num_funcs, alpha):
        return make_pipeline(
            KBinsDiscretizer(n_bins=num_funcs, encode='onehot', strategy=strategy),
            Ridge(alpha=alpha)
        )

In [61]:
class StrumLiouvilleHPO:
    def search_space(self, trial: optuna.Trial):
        curvature_gamma = trial.suggest_float('curvature_gamma', 1e-3, 2, log=True)
        spline_degree = trial.suggest_int('spline_degree', 1, 5)
        knots = trial.suggest_categorical('knots', ['uniform', 'quantile'])
        num_funcs = trial.suggest_int('num_funcs', 1 + spline_degree, 10)
        alpha = trial.suggest_float('alpha', 1e-3, 1e3, log=True)

        return {
            'curvature_gamma': curvature_gamma,
            'spline_degree': spline_degree,
            'knots': knots,
            'num_funcs': num_funcs,
            'alpha': alpha
        }

    def pipeline(self, curvature_gamma, spline_degree, knots, num_funcs, alpha):
        spline_n_knots = num_funcs - (spline_degree - 1)
        return make_pipeline(
            make_column_transformer(
                (
                    SplineTransformer(n_knots=spline_n_knots, knots=knots, degree=spline_degree),
                    float_feat_cols
                ),
                (
                    StrumLiouvilleTransformer(num_funcs=num_funcs, curvature_gamma=curvature_gamma),
                    int_feat_cols
                )
            ),
            Ridge(alpha=alpha)
        )

In [56]:
def hpo(pipeline_hpo, n_trials=50):
    def objective(trial: optuna.Trial):
        params = pipeline_hpo.search_space(trial)
        model = pipeline_hpo.pipeline(**params)
        scores = cross_val_score(model, X, y, cv=5, scoring='neg_root_mean_squared_error')
        return -np.mean(scores)

    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=n_trials, show_progress_bar=True)

    best_params = study.best_params
    print("Best params:", best_params)
    print("Best CV RMSE:", study.best_value)

    best_model = pipeline_hpo.pipeline(**best_params)
    best_model.fit(X_train, y_train)
    test_rmse = root_mean_squared_error(y_test, best_model.predict(X_test))
    print("Test RMSE:", test_rmse)
    return best_model

In [52]:
hpo(SplineHpo())

[I 2025-09-01 09:47:19,855] A new study created in memory with name: no-name-64316a42-c94b-4dc8-9d84-fcd5df3fab1e


  0%|          | 0/50 [00:00<?, ?it/s]

[I 2025-09-01 09:47:20,133] Trial 0 finished with value: 11022.832437439907 and parameters: {'knots': 'quantile', 'degree': 3, 'num_funcs': 9, 'alpha': 100.87560512554876}. Best is trial 0 with value: 11022.832437439907.
[I 2025-09-01 09:47:20,342] Trial 1 finished with value: 11046.96039922694 and parameters: {'knots': 'quantile', 'degree': 2, 'num_funcs': 8, 'alpha': 0.004413421199052658}. Best is trial 0 with value: 11022.832437439907.
[I 2025-09-01 09:47:20,482] Trial 2 finished with value: 11051.851641212164 and parameters: {'knots': 'uniform', 'degree': 2, 'num_funcs': 5, 'alpha': 0.41588029060062337}. Best is trial 0 with value: 11022.832437439907.
[I 2025-09-01 09:47:20,720] Trial 3 finished with value: 11056.048968718995 and parameters: {'knots': 'quantile', 'degree': 3, 'num_funcs': 8, 'alpha': 0.002540905660227689}. Best is trial 0 with value: 11022.832437439907.
[I 2025-09-01 09:47:20,948] Trial 4 finished with value: 11039.217369614698 and parameters: {'knots': 'quantile',

,steps,"[('splinetransformer', ...), ('ridge', ...)]"
,transform_input,None
,memory,None
,verbose,False
,n_knots,10
,degree,1
,knots,'quantile'
,extrapolation,'constant'
,include_bias,True
,order,'C'
,sparse_output,False


In [57]:
hpo(BinsHPO())

[I 2025-09-01 09:48:10,191] A new study created in memory with name: no-name-ac5254ca-e31a-4744-ad78-10b17dfd5219


  0%|          | 0/50 [00:00<?, ?it/s]

[I 2025-09-01 09:48:10,324] Trial 0 finished with value: 11099.962885955587 and parameters: {'strategy': 'uniform', 'num_funcs': 3, 'alpha': 824.8824150425572}. Best is trial 0 with value: 11099.962885955587.
[I 2025-09-01 09:48:10,426] Trial 1 finished with value: 11058.878672718281 and parameters: {'strategy': 'quantile', 'num_funcs': 2, 'alpha': 0.08995371012315029}. Best is trial 1 with value: 11058.878672718281.
[I 2025-09-01 09:48:10,612] Trial 2 finished with value: 11122.196079204174 and parameters: {'strategy': 'uniform', 'num_funcs': 9, 'alpha': 14.690434014855715}. Best is trial 1 with value: 11058.878672718281.
[I 2025-09-01 09:48:10,754] Trial 3 finished with value: 11022.523696104694 and parameters: {'strategy': 'quantile', 'num_funcs': 7, 'alpha': 0.007907337432291255}. Best is trial 3 with value: 11022.523696104694.
[I 2025-09-01 09:48:10,894] Trial 4 finished with value: 11105.080754433944 and parameters: {'strategy': 'uniform', 'num_funcs': 5, 'alpha': 35.442836481949

,steps,"[('kbinsdiscretizer', ...), ('ridge', ...)]"
,transform_input,None
,memory,None
,verbose,False
,n_bins,8
,encode,'onehot'
,strategy,'quantile'
,quantile_method,'warn'
,dtype,None
,subsample,200000
,random_state,None


In [62]:
hpo(StrumLiouvilleHPO())

[I 2025-09-01 09:51:18,369] A new study created in memory with name: no-name-c80b10e2-c575-4236-b879-099ca477b24d


  0%|          | 0/50 [00:00<?, ?it/s]

[I 2025-09-01 09:51:28,040] Trial 0 finished with value: 11067.26919512454 and parameters: {'curvature_gamma': 0.013261522194772286, 'spline_degree': 5, 'knots': 'uniform', 'num_funcs': 9, 'alpha': 0.0040365370275534315}. Best is trial 0 with value: 11067.26919512454.
[I 2025-09-01 09:51:38,669] Trial 1 finished with value: 11078.02231917473 and parameters: {'curvature_gamma': 0.043021417256202676, 'spline_degree': 5, 'knots': 'uniform', 'num_funcs': 10, 'alpha': 232.28440870304428}. Best is trial 0 with value: 11067.26919512454.
[I 2025-09-01 09:51:47,337] Trial 2 finished with value: 11053.761591324657 and parameters: {'curvature_gamma': 0.170997081153672, 'spline_degree': 5, 'knots': 'uniform', 'num_funcs': 8, 'alpha': 6.870852431210281}. Best is trial 2 with value: 11053.761591324657.
[I 2025-09-01 09:51:55,150] Trial 3 finished with value: 11032.653046720525 and parameters: {'curvature_gamma': 0.017199000944498173, 'spline_degree': 3, 'knots': 'quantile', 'num_funcs': 7, 'alpha': 

,steps,"[('columntransformer', ...), ('ridge', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('splinetransformer', ...), ('strumliouvilletransformer', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True
